In [1]:
import kickstart as ks
dataset = ks.load_data("digits_train.jsonlines")

In [2]:
print "Cantidad de muestras: ", len(dataset)
print dataset[0].keys()

Cantidad de muestras:  60000
[u'image', u'label']


In [3]:
from itertools import islice

def draw_ascii_image(image):
    it = iter(image)
    for _ in range(28):
        line = []
        for b in islice(it, 28):
            if b < 80:
                c = "."
            elif b < 160:
                c = "*"
            else:
                c = "#"
            line.append(c * 2)
        print("".join(line))

Mezclamos el dataset para distribuirlo de mejor manera.

In [4]:
from random import shuffle
shuffle(dataset)

Vemos cómo están formados los dígitos.

In [5]:
print "Dígito de muestra: ", dataset[0]['label']
draw_ascii_image(dataset[0]['image'])

Dígito de muestra:  5
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
..........................########################......
..........................########################......
......................############################......
....................##############################......
....................##############......................
....................##############......................
....................##################..................
....................####################................
....................####################................
................................########................
.........

Separar sets en inputs y labels.

In [11]:
import pandas as pd
dataset = pd.DataFrame(dataset)

X = dataset['image']
y = dataset['label']

Separar train y valid sets usando métodos de sklearn.

In [68]:
from sklearn.cross_validation import StratifiedKFold

eval_size = .25
kf = StratifiedKFold(y, round(1./eval_size))
train_indices, valid_indices = next(iter(kf))

Usar este split si se desea usar cross_val_score (porque el StratifiedKFold se hace dentro).

In [75]:
X_train = [sample for sample in X]
y_train = [label for label in y]

Usar este split si se usa otra validacion (los indices los devuelve StratifiedKFold).

In [69]:
X_train = [sample for sample in X[train_indices]]
y_train = [label for label in y[train_indices]]

In [ ]:
Otro metodo para hacer el split, este mantiene la forma de los sets (pandas.DataFrames).

In [ ]:
#X_train, y_train = X[train_indices], y[train_indices]
#X_valid, y_valid = X[valid_indices], y[valid_indices]

In [14]:
print "Cantidad de muestras en el set de entrenamiento: ", len(X_train)
#print "Cantidad de muestras en el set de validación: ", len(X_valid)

Cantidad de muestras en el set de entrenamiento:  44995
Cantidad de muestras en el set de validación:  15005


Una vez separados, podemos utilizar RandomForests o DecisionTrees para entrenar el modelo y predecir sobre el set de validacion.

In [15]:
import numpy as np
from sklearn.cross_validation import cross_val_score

In [66]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
#rf.fit(X_train.T.values, y_train.values)

ValueError: setting an array element with a sequence.

In [70]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
#dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [62]:
X_train = np.array(X_train.T)
y_train = np.array(y_train)

In [101]:
s = cross_val_score(rf, X_train, y=y_train, cv=10, n_jobs=2)

In [102]:
print "Accuracy: ", np.mean(s)

Accuracy:  0.945316624861


In [76]:
s = cross_val_score(dt, X_train, y=y_train, cv=10, n_jobs=2)

In [77]:
print "Accuracy: ", np.mean(s)

Accuracy:  0.873549205226


Transformo los inputs sets con PCA.

In [96]:
from sklearn.decomposition import RandomizedPCA, PCA

X_train_pca = np.array(X_train)

nc = 60
#pca = RandomizedPCA(n_components=nc).fit(X_train_pca)
pca = PCA(n_components=nc).fit(X_train_pca)
X_train_pca = pca.transform(X_train_pca)

In [97]:
s = cross_val_score(dt, X_train_pca, y=y_train, cv=10, n_jobs=2)

In [100]:
print "Accuracy: %.2f +/-%.2f" %(s.mean(), s.std()*2)

Accuracy: 0.84 +/-0.01


Vamos a predecir y calcular accuracy con sklearn.

In [105]:
X_train_s = [sample for sample in X[train_indices]]
y_train_s = [label for label in y[train_indices]]
X_valid_s = [sample for sample in X[valid_indices]]
y_valid_s = [label for label in y[valid_indices]]

In [107]:
dt = DecisionTreeClassifier()
dt.fit(X_train_s, y_train_s)
pred = dt.predict(X_valid_s)

In [108]:
from sklearn.metrics import accuracy_score
print "Accuracy: ", accuracy_score(y_valid_s, pred)

Accuracy:  0.866577807398


Ahora voy a calcular accuracy pero haciendo PCA.

In [109]:
nc = 60
X_train_s_pca = np.array(X_train_s)
pca = PCA(n_components=nc).fit(X_train_s_pca)
X_train_s_pca = pca.transform(X_train_s_pca)

In [110]:
dt.fit(X_train_s_pca, y_train_s)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [111]:
X_valid_s_pca = pca.transform(X_valid_s)

In [112]:
pred = dt.predict(X_valid_s_pca)

In [113]:
from sklearn.metrics import accuracy_score
print "Accuracy: ", accuracy_score(y_valid_s, pred)

Accuracy:  0.832455848051
